In [25]:
using HDF5
p::Array{Float64} = Float64[]
t::Array{Float64} = Float64[]
u::Array{Float64} = Float64[]
v::Array{Float64} = Float64[]
x::Array{Float64} = Float64[]
y::Array{Float64} = Float64[]
Bx::Array{Float64} = Float64[]
By::Array{Float64} = Float64[]

h5open("../sims/mhd/mhd_rk4_naive_result.hdf5", "r") do file
    global p = read(file, "result")["value"]["p"]["value"]
    global t = read(file, "result")["value"]["t"]["value"]
    global u = read(file, "result")["value"]["u"]["value"]
    global v = read(file, "result")["value"]["v"]["value"]
    global x = read(file, "result")["value"]["x"]["value"]
    global y = read(file, "result")["value"]["y"]["value"]
    global Bx = read(file, "result")["value"]["Bx"]["value"]
    global By = read(file, "result")["value"]["By"]["value"]
end

h5open("../sims/mhd/mhd_rk4_naive_result.hdf5", "r")

🗂️ HDF5.File: (read-only) ../sims/mhd/mhd_rk4_naive_result.hdf5
└─ 📂 result
   ├─ 🏷️ OCTAVE_NEW_FORMAT
   ├─ 🔢 type
   └─ 📂 value
      ├─ 📂 Bx
      │  ├─ 🏷️ OCTAVE_NEW_FORMAT
      │  ├─ 🔢 type
      │  └─ 🔢 value
      ├─ 📂 By
      │  ├─ 🏷️ OCTAVE_NEW_FORMAT
      │  ├─ 🔢 type
      │  └─ 🔢 value
      ├─ 📂 p
      │  ├─ 🏷️ OCTAVE_NEW_FORMAT
      │  ├─ 🔢 type
      │  └─ 🔢 value
      ├─ 📂 t
      │  ├─ 🏷️ OCTAVE_NEW_FORMAT
      │  ├─ 🔢 type
      │  └─ 🔢 value
      ├─ 📂 u
      │  ├─ 🏷️ OCTAVE_NEW_FORMAT
      │  ├─ 🔢 type
      │  └─ 🔢 value
      ├─ 📂 v
      │  ├─ 🏷️ OCTAVE_NEW_FORMAT
      │  ├─ 🔢 type
      │  └─ 🔢 value
      ├─ 📂 x
      │  ├─ 🏷️ OCTAVE_NEW_FORMAT
      │  ├─ 🔢 type
      │  └─ 🔢 value
      └─ 📂 y
         ├─ 🏷️ OCTAVE_NEW_FORMAT
         ├─ 🔢 type
         └─ 🔢 value

In [26]:
include("../functions/GreedyRegression.jl")
include("../functions/Integrate.jl")
include("../functions/ReportModel.jl")
include("../functions/EnvelopePol.jl")
include("../functions/PickSubdomains.jl")

using .GreedyRegression, .Integrate, .ReportModel, .EnvelopePol, .PickSubdomains

nl::Int = 2 # under-estimate this
nw::Int = 128 # number of domains we integrate over 
dof::Int = 1 # scalars have one degree of freedom
dim::Int = 3 # how many dimensions does our data have?
env_pow::Int = 4 # weight is (1-x^2)^power
size_vec::Vector{Int} = [64,64,32] # how many gridpoints should we use per integration?
buffer::Int = 0 # Don't use points this close to boundary

pol::Matrix{Float64} = EnvelopePol.envelope_pol(env_pow, dim)

size_of_data::Vector{Int} = [size(u,1), size(u,2), size(u,3)]
seed::Int = 1
corners::Matrix{Int} = PickSubdomains.pick_subdomains(size_of_data, size_vec, buffer, nw, seed)

grid::Vector{Vector{Float64}} = [vec(y), vec(x), vec(t)]
corners

3×128 Matrix{Int64}:
  5  23  45  41  59  13  50  50  43  …  20  54  58  10  51  44  36  45  24
 64   3  52   8  53  39  12  34   8     56  23   7  30   1   6  13   9  40
 36  21   5   2  28  31  44   4  57     52   3  68  45  23  15  56  41  68

In [27]:
using Statistics

G::Matrix{Float64} = zeros(Float64, dof*nw, 0)
labels::Vector{String} = String[]
scales::Vector{Float64} = Float64[]

function add_library_term(label::AbstractString, data::AbstractArray, derivs::AbstractVector{Int64}, scale::AbstractFloat)
    col::Vector{Float64} = Integrate.integrate(data, derivs, grid, corners, size_vec, pol)

    global G = hcat(G, col)
    push!(labels, label)
    push!(scales, scale)
end

# flow terms
add_library_term("∂u/∂x", u, [2], 1.)
add_library_term("∂v/∂x", v, [2], 1.)
add_library_term("∂u/∂y", u, [1], 1.)
add_library_term("∂v/∂y", v, [1], 1.)
add_library_term("∂u/∂t", u, [3], 1.)
add_library_term("∂v/∂t", v, [3], 1.)
add_library_term("∂u^2/∂x", u .* u, [2], 1.)
add_library_term("∂uv/∂x", u .* v, [2], 1.)
add_library_term("∂uv/∂y", u .* v, [1], 1.)
add_library_term("∂v^2/∂y", v .* v, [1], 1.)

# flow terms second derivatives
add_library_term("∂^2u/∂x^2", u, [2,2], 1.)
add_library_term("∂^2u/∂y^2", u, [1,1], 1.)
add_library_term("∂^2v/∂x^2", v, [2,2], 1.)
add_library_term("∂^2v/∂y^2", v, [1,1], 1.)

# pressure terms
add_library_term("∂P/∂x", p, [2], 1.)
add_library_term("∂P/∂y", p, [1], 1.)

# B field terms
add_library_term("∂Bx^2/∂x", Bx .* Bx, [2], 1.)
add_library_term("∂Bx^2/∂y", Bx .* Bx, [1], 1.)
add_library_term("∂By^2/∂x", By .* By, [2], 1.)
add_library_term("∂By^2/∂y", By .* By, [1], 1.)
add_library_term("∂BxBy/∂x", Bx .* By, [2], 1.)
add_library_term("∂BxBy/∂y", Bx .* By, [1], 1.)
add_library_term("∂Bx/∂t", Bx, [3], 1.)
add_library_term("∂By/∂t", By, [3], 1.)

# B field second derivatives
add_library_term("∂^2Bx/∂x^2", Bx, [2, 2], 1.)
add_library_term("∂^2By/∂x^2", By, [2, 2], 1.)
add_library_term("∂^2Bx/∂y^2", Bx, [1, 1], 1.)
add_library_term("∂^2By/∂y^2", By, [1, 1], 1.)

# weird terms
add_library_term("∂uBy/∂x", u .* By, [2], 1.)
add_library_term("∂vBx/∂x", v .* Bx, [2], 1.)
add_library_term("∂uBy/∂y", u .* By, [1], 1.)
add_library_term("∂vBx/∂y", v .* Bx, [1], 1.)

norm_vec::Vector{Float64} = Integrate.integrate(0*u .+ 1, Int64[], grid, corners, size_vec, pol)
G ./= norm_vec
G ./= scales'

G

128×32 Matrix{Float64}:
 -0.00353846    0.171212    -0.162234    …  -0.0150068     0.00600844
 -0.0181185    -0.140253     0.137326       -0.127717      0.0303346
 -0.0144542    -0.0728453    0.125125        0.0431232    -0.0100712
  0.0298757    -0.00448689  -0.0472018      -0.00150894   -0.000366356
 -0.0485808    -0.165116     0.196651        0.0639878    -0.0561529
 -0.111748     -0.0225108   -0.0236538   …  -0.0167311     0.0939198
  8.18071e-5   -0.0305982    0.0651341      -0.04056       0.00584842
  0.00918382   -0.00366316   0.00249152      0.0185964     0.0157029
  0.0617242    -0.0924338    0.073228       -0.0659049     0.0207098
 -0.0355838     0.213899    -0.221283       -0.11205       0.0265496
 -0.110242      0.0105893   -0.00648751  …   0.000399273  -0.011339
  0.142874      0.025939    -0.013445        0.0201993    -0.0235676
 -0.176889      0.00214238   0.00462654     -0.0147398     0.114966
  ⋮                                      ⋱   ⋮            
 -0.0187875    -0.

In [28]:
using LinearAlgebra

gamma::Float64 = 1.5

for i::Int = 1:5
    cs::Matrix{Float64}, residuals::Vector{Float64}, _ = GreedyRegression.greedy_regression(G)
    k::Int = ReportModel.report_identified_model(cs, residuals, scales, labels, gamma)

    col_norms::Vector{Float64} = [norm(G[:, j] .* cs[j, k]) for j = 1:size(G, 2)]
    kill::Int = argmax(col_norms)
    G = G[:, setdiff(1:end, kill)]
    labels = labels[setdiff(1:end, kill)]
    scales = scales[setdiff(1:end, kill)]
end

∂u/∂x + ∂v/∂y = 0
∂u/∂t + ∂u^2/∂x + 0.99∂uv/∂y - 0.013∂^2u/∂x^2 - 0.01∂^2u/∂y^2 + ∂P/∂x - ∂Bx^2/∂x - ∂BxBy/∂y = 0
∂v/∂t + ∂uv/∂x + ∂v^2/∂y - 0.011∂^2v/∂x^2 - 0.012∂^2v/∂y^2 + 0.99∂P/∂y - 0.99∂By^2/∂y - 0.99∂BxBy/∂x = 0
∂Bx/∂t - 0.019∂^2Bx/∂x^2 - 0.021∂^2Bx/∂y^2 - ∂uBy/∂y + ∂vBx/∂y = 0
∂By/∂t - 0.021∂^2By/∂x^2 - 0.019∂^2By/∂y^2 + ∂uBy/∂x - ∂vBx/∂x = 0
